In [97]:
from networkx import maximum_flow
from networkx import gnm_random_graph
from networkx import draw 
from networkx import adjacency_matrix
from networkx import all_simple_paths
from networkx import shortest_path
from networkx import set_edge_attributes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

# Graph

In [98]:
min_weight = 5
max_weight = 15
n_nodes = 5 * 10 ** 4
n_edges = 10 ** 6
n_nodes = 5 * 10 ** 2
n_edges = 10 ** 4
n_nodes = 5
n_edges = 20
# n_nodes, n_edges = 50000, 1000000 # <|-- takes 17.5 s to compute the maximum flow in a graph
source, destination = 0, 1 # don't need to randomize (graph is randomized)

## Creates a random valuated graph

In [99]:
def modify_weights_graph (Graph, new_weights) :
    """ utility function for setting weights (dual problem of shortest path) """
    set_edge_attributes(
        Graph,
        values = dict({edge : dual for edge, dual in zip (Graph.edges, new_weights)}),
        name = 'weight'
    )

In [100]:
Graph = gnm_random_graph(n_nodes, n_edges, directed=True)
Graph_dual = deepcopy(Graph)

for (u, v) in Graph.edges():
    Graph.edges[u,v]['weight'] = np.random.uniform(min_weight, max_weight)

In [101]:
Adj_matrix = adjacency_matrix(Graph)
Adj_matrix.shape

C:\Users\val\AppData\Local\Temp\ipykernel_15948\2680887831.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj_matrix = adjacency_matrix(Graph)


(5, 5)

In [102]:
flow, flow_dict = maximum_flow(Graph, _s = source, _t = destination, capacity='weight')
# total flow, flow first node
flow, flow_dict[0]

(44.83289938518128,
 {1: 14.325785489942257,
  2: 7.589312015460586,
  3: 12.516568714938677,
  4: 10.401233164839757})

Maximum flow is equivalent to maximum path generation **if we consider all possible paths**

# Column Generation

In [103]:
import pulp as pl

In [104]:
solver = pl.getSolver('CPLEX_CMD', timeLimit=10)
solver.path

'cplex.exe'

In [105]:
def get_n_paths (Graph, source, destination, max_iter = 16):
    """ Get the max_iter paths of all_simple_paths generator: Graph: source --|> destination """
    list_ = []
    for (index, path) in enumerate(all_simple_paths(Graph, source, destination, cutoff=10)):
        list_.append(path)
        if (index >= max_iter - 1):
            return (list_)
    return (list_)

In [107]:
# affichage de n chemins 
len(get_n_paths(Graph, source, destination, max_iter=50))

16

## Master

In [108]:
# Etape 0 INIT THE MASTER PROBLEM

lpProb = pl.LpProblem("path_generation", pl.LpMaximize)
lpProb += 0 # initiate the objective function to 0 (necessary line of code)

In [109]:
# useful to know the final paths at the end
path2idx = {}
path_count = 0

In [110]:
def add_paths (lpProb, paths, path_count, path2idx):
    # pour chaque path ajouter une variable et ses coef. dans les contraintes
    for path in paths :

        # add a var path
        prov = pl.LpVariable("X"+str(path_count), 0, None, pl.LpContinuous)
        lpProb.objective.addterm(prov, 1)
        
        # for each edge on the path, search the associated constraint (indexed by edge)
        for index in range(len(path)-1) :
            index_ctr = str((path[index], path[index + 1]))
            if index_ctr in lpProb.constraints:
                lpProb.constraints[index_ctr].addterm(prov, 1) # add term in constraint
            else:
                lpProb.constraints[index_ctr] = prov <= Graph.edges[(path[index], path[index + 1])]
        
        path2idx[path_count] = str(path)
        path_count += 1
    return (path_count, path2idx)

In [111]:
path_count, path2idx = add_paths (lpProb, get_n_paths (Graph, source, destination, max_iter = 18), path_count, path2idx)

In [112]:
# solver = pl.CPLEX_PY()
result = lpProb.solve()
# result = lpProb.solve(pl.GUROBI_CMD(options=[("MIPgap", 0.9)]))

In [113]:
# affichage du MASTER PROBLEM
lpProb

path_generation:
MAXIMIZE
1*X0 + 1*X1 + 1*X10 + 1*X11 + 1*X12 + 1*X13 + 1*X14 + 1*X15 + 1*X2 + 1*X3 + 1*X4 + 1*X5 + 1*X6 + 1*X7 + 1*X8 + 1*X9 + 0
SUBJECT TO
(0, 1): X0 <= 14.3257854899

(0, 2): X1 + X2 + X3 + X4 + X5 <= 7.58931201546

(2, 1): X1 + X10 + X12 + X15 + X7 <= 10.0947274599

(2, 3): X13 + X2 + X3 <= 7.80422780634

(3, 1): X13 + X14 + X2 + X5 + X6 <= 10.4044050004

(3, 4): X10 + X3 + X9 <= 10.1186350608

(4, 1): X11 + X3 + X4 + X8 + X9 <= 10.0079814349

(2, 4): X4 + X5 + X8 <= 12.7523626368

(4, 3): X14 + X15 + X5 <= 8.67067380674

(0, 3): X10 + X6 + X7 + X8 + X9 <= 12.7314078661

(3, 2): X15 + X7 + X8 <= 5.14924547057

(4, 2): X10 + X12 + X13 <= 5.9211499804

(0, 4): X11 + X12 + X13 + X14 + X15 <= 10.4012331648

VARIABLES
X0 Continuous
X1 Continuous
X10 Continuous
X11 Continuous
X12 Continuous
X13 Continuous
X14 Continuous
X15 Continuous
X2 Continuous
X3 Continuous
X4 Continuous
X5 Continuous
X6 Continuous
X7 Continuous
X8 Continuous
X9 Continuous

## Dual variables

In [114]:
def get_duals(lpPb):
    """ Get the dual variables from a problem Pulp """
    return [c.pi for _, c in list(lpPb.constraints.items())]

In [115]:
get_duals(lpProb)[0:8]

[1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0]

## Iterative procedure

In [121]:
go_on = True

# while go_on :
for i in range(1):

    duals = get_duals(lpProb)

    modify_weights_graph (Graph_dual, duals)

    path_q = shortest_path(Graph_dual, source, destination)

    reduced_cost = 1

    for e in range(len(path_q)-1) :
        try :
            reduced_cost -= lpProb.constraints[str((path_q[e], path_q[e + 1]))].pi
        except KeyError : # <|-- constraint does not exist (so dual == 0)
            pass

    if reduced_cost >= 0 : # add path q in lp Problem
        path_count, path2idx = add_paths(lpProb, [path_q], path_count, path2idx)
    else :
        go_on = False


In [122]:
reduced_cost

0.0

In [123]:
go_on

True

In [124]:
lpProb

path_generation:
MAXIMIZE
1*X0 + 1*X1 + 1*X10 + 1*X11 + 1*X12 + 1*X13 + 1*X14 + 1*X15 + 1*X16 + 1*X17 + 1*X18 + 1*X19 + 1*X2 + 1*X3 + 1*X4 + 1*X5 + 1*X6 + 1*X7 + 1*X8 + 1*X9 + 0
SUBJECT TO
(0, 1): X0 + X16 + X17 + X18 + X19 <= 14.3257854899

(0, 2): X1 + X2 + X3 + X4 + X5 <= 7.58931201546

(2, 1): X1 + X10 + X12 + X15 + X7 <= 10.0947274599

(2, 3): X13 + X2 + X3 <= 7.80422780634

(3, 1): X13 + X14 + X2 + X5 + X6 <= 10.4044050004

(3, 4): X10 + X3 + X9 <= 10.1186350608

(4, 1): X11 + X3 + X4 + X8 + X9 <= 10.0079814349

(2, 4): X4 + X5 + X8 <= 12.7523626368

(4, 3): X14 + X15 + X5 <= 8.67067380674

(0, 3): X10 + X6 + X7 + X8 + X9 <= 12.7314078661

(3, 2): X15 + X7 + X8 <= 5.14924547057

(4, 2): X10 + X12 + X13 <= 5.9211499804

(0, 4): X11 + X12 + X13 + X14 + X15 <= 10.4012331648

VARIABLES
X0 Continuous
X1 Continuous
X10 Continuous
X11 Continuous
X12 Continuous
X13 Continuous
X14 Continuous
X15 Continuous
X16 Continuous
X17 Continuous
X18 Continuous
X19 Continuous
X2 Continuous
X3 Contin